# Workshop Building an AI Agent

### Install all necessary dependencies. Select Python 3.13

In [103]:
%pip install -r requirements.txt > /dev/null

Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


### Load dependencies from helper file.

In [104]:
# Load environment variables from .env
%run -i helper.py

### Initialize Language Model

In [105]:
llm = init_llm('gpt-4o', max_tokens=2048, temperature=0)
# llm = init_llm('gemini-2.5-pro', max_tokens=4192, temperature=0, init_func=google_vertexai_init_chat_model)

llm.invoke("What day is it?").content


"I don't have real-time capabilities to provide the current date. You can check the date on your device or calendar."

### Exercise
**Exercise:** Give the Agent a function `get_today()` and invoke it asking about the date.

<details>
<summary>Show solution</summary>

```python
@tool
def get_today() -> str:
    "This tool retrieves the current date and time."
    return date.today()

tools = [get_today]
tool_llm = llm.bind_tools(tools)
answer = tool_llm.invoke("What's todays date?")

display(f"AI Response: {answer.content}" if answer.content else "No AI Message.")
display(f"AI Actions: {answer.additional_kwargs}")

In [106]:
#Implement the function
def get_today():
    """TODO: Define this docstring."""
    pass

#Add the tool
tools = []

agent_llm = llm.bind_tools(tools)

#Add message for the agent
answer = agent_llm.invoke("")

display(f"AI Response: {answer.content}" if answer.content else "No AI Message.")
display(f"AI Actions: {answer.additional_kwargs}")


'No AI Message.'

'AI Actions: {\'tool_calls\': [{\'id\': \'call_2bQpYDttu7ymj4ijIpYpJQCh\', \'function\': {\'arguments\': \'{"location": "New York"}\', \'name\': \'get_current_weather\'}, \'type\': \'function\'}, {\'id\': \'call_5hEEWCvZhtLolZq6CVsAcIey\', \'function\': {\'arguments\': \'{"location": "Los Angeles"}\', \'name\': \'get_current_weather\'}, \'type\': \'function\'}], \'refusal\': None}'

### SOLUTION

In [107]:
@tool
def get_today() -> str:
    "This tool retrieves the current date and time."
    return date.today()

tools = [get_today]
tool_llm = llm.bind_tools(tools)
answer = tool_llm.invoke("What's todays date?")

display(f"AI Response: {answer.content}" if answer.content else "No AI Message.")
display(f"AI Actions: {answer.additional_kwargs}")

'No AI Message.'

"AI Actions: {'tool_calls': [{'id': 'call_7Q1urEZEoNR4GwNEpWsY0sRu', 'function': {'arguments': '{}', 'name': 'get_today'}, 'type': 'function'}], 'refusal': None}"

### Build a ReAct agent (LLM with tools to ReAct Agent)
Now we implement tool execution.

In [108]:
from langgraph.prebuilt import create_react_agent

tools = [get_today]
react_agent = create_react_agent(model=llm, tools=tools)
answer = react_agent.invoke({"messages": ["What's todays date?"]})["messages"]
print(answer[-1].content)

Today's date is September 26, 2025.


Stream the agents output

In [109]:
display_graph(react_agent)
for msg in answer:
    msg.pretty_print()



================================ Human Message =================================

What's todays date?
================================== Ai Message ==================================
Tool Calls:
  get_today (call_lQN1b1IIzJnPK6DT0DH9238S)
 Call ID: call_lQN1b1IIzJnPK6DT0DH9238S
  Args:
================================= Tool Message =================================
Name: get_today

2025-09-26
================================== Ai Message ==================================

Today's date is September 26, 2025.


### Giving tools to our LangGraph Agent
...
**The Critical Role of Docstrings**

For a LangGraph agent to effectively use a custom function as a "tool," it must first understand what the function does. The primary source of this understanding for the agent is the function's docstring. The LLM parses the docstring to learn the tool's purpose, its required parameters, and what kind of output to expect. A well-written docstring is therefore essential for the agent to know when and how to use the tool correctly.

### Exercise
**Documenting an Agent Tool**

In the next cell, you will find a `get_records()` function designed to fetch data from an API. Your task is to understand the function and complete its docstring. 

(This exercise highlights how crucial clear documentation is, as it directly enables the agent to correctly utilize the function to fulfill user requests.)

<details>
<summary>Show solution </summary>

```python

odata_endpoint = os.getenv("ODATA_ENDPOINT")
if not odata_endpoint:
    raise ValueError("ODATA_ENDPOINT environment variable is not set.")

@tool
def get_records(top:int) -> str:
    """Fetch ExternalTimeData records for the current user.
    Args:
        top: Maximum number of records to return.
    Returns:
        JSON response with time records or error message.
    """
    # Built query
    params = {}
    
    # Limit number of records returned
    if top is not None:
        params['$top'] = top

    # Filter by current userID
    params['$filter'] = f'userId eq {user_id}'
    query_text = urllib.parse.urlencode(params, safe='(),')

    # Make GET request to the SuccessFactors OData API
    table = 'ExternalTimeData'
    url = f'{odata_endpoint}{table}?{query_text}'
    response = requests.get(url, headers=get_header)

    # Process response
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return f'Error: {response.content}'

In [110]:
@tool
def get_records(top:int) -> str:
    """TODO Understand the function and fill in the docstring.
    """
    # Built query
    params = {}
    
    # Limit number of records returned
    if top is not None:
        params['$top'] = top

    # Filter by current userID
    params['$filter'] = f'userId eq {user_id}'
    query_text = urllib.parse.urlencode(params, safe='(),')

    # Make GET request to the SuccessFactors OData API
    table = 'ExternalTimeData'
    url = f'{odata_endpoint}{table}?{query_text}'
    response = requests.get(url, headers=get_header)

    # Process response
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return f'Error: {response.content}'

### Solution

In [111]:
#SuccessFactors OData API endpoint to fetch time records for the current user.
odata_endpoint = os.getenv("ODATA_ENDPOINT")
if not odata_endpoint:
    raise ValueError("ODATA_ENDPOINT environment variable is not set.")
    
@tool
def get_records(top:int) -> str:
    """Fetch ExternalTimeData records for the current user.
    Args:
        top: Maximum number of records to return.
    Returns:
        JSON response with time records or error message.
    """
    # Built query
    params = {}
    
    # Limit number of records returned
    if top is not None:
        params['$top'] = top

    # Filter by current userID
    params['$filter'] = f'userId eq {user_id}'
    query_text = urllib.parse.urlencode(params, safe='(),')

    # Make GET request to the SuccessFactors OData API
    table = 'ExternalTimeData'
    url = f'{odata_endpoint}{table}?{query_text}'
    response = requests.get(url, headers=get_header)

    # Process response
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return f'Error: {response.content}'

### Built a first version of our agent
Update the agent's tools and ask for the latest records.

In [112]:
tools = [get_today, get_records]

react_agent = create_react_agent(model=llm, tools=tools)
answer = react_agent.invoke({"messages": ["Get me the latest time records."]})["messages"]
print(answer[-1].content)

There are no time records available.


No records so far. Let's add some.

### Add a tool for adding records

In [113]:
#LangGraph requires typing information for structured data inputs/outputs. Define a BaseModel for the input data.
class ExternalTimeData(BaseModel):
    """Represents a single time record."""
    startDate: str = Field(description="The date of the work period. Example: 2023-09-15")
    startTime: str = Field(description="The start time of the work period in ISO 8601 format. Example: PT09H00M00S")
    endTime: str = Field(description="The end time of the work period in ISO 8601 format. Example: PT17H00M00S")

In [114]:
@tool
def post_records(data: ExternalTimeData, confirmation_message : str):
    """Post a single work time record to the API. The user is asked for confirmation first.

    Args:
        data (ExternalTimeData): An object with the details for the time record. It must contain:
            - startDate (str): The work date in 'YYYY-MM-DD' format.
            - startTime (str): The work start time in ISO 8601 duration format (e.g., 'PT09H00M00S').
            - endTime (str): The work end time in ISO 8601 duration format (e.g., 'PT17H30M00S').
        confirmation_message (str): A detailed summary for the user to confirm, including the specific date and time range.
            Example: 'Confirm work time from 09:00 to 17:30 on Friday, September 26th 2025.'
    """
    # Build POST payload
    payload, url = build_post_payload(data, odata_endpoint)
    # Make POST request to the SuccessFactors OData API endpoint
    response = requests.post(url, headers=post_header, json=payload)

    # Process response
    if response.status_code in (200, 201):
        return f'Entity created successfully: {data}'
    else:
        return  f'Error creating entity ({data}):  {response.status_code}: {response.content}'

### Exercise
**Preparing Tool Inputs**

An agent needs to know not just *what* a tool does, but also *how* to provide the correct inputs. The `post_records` function requires data in a specific format to log time entries.

**Your Task:**

In the next cell, you'll find an `example_record`. Examine the `post_records` function and its docstring to understand the required data structure. Then, fill in the `example_record` with valid data. This exercise simulates how the agent prepares data before calling a tool.

<details>
<summary>Show solution</summary>

```python
# Create example record like the agent would
example_record: ExternalTimeData = {
    "startDate": "2025-01-01",
    "startTime": "PT09H00M00S",
    "endTime": "PT18H30M00S"
}

In [115]:
# TODO Create example record like the agent would
example_record = ExternalTimeData(
    startDate="",
    startTime="",
    endTime=""
)
# Uncomment to try out
#post_records.invoke({"data": example_record, "confirmation_message": "No confirmation implemented yet."})

### Solution

In [116]:
# Create example record like the agent would
example_record = ExternalTimeData(
    startDate="2025-01-01",
    startTime="PT09H00M00S",
    endTime="PT18H30M00S"
)

post_records.invoke({"data": example_record, "confirmation_message": "No confirmation implemented yet."})

"Entity created successfully: startDate='2025-01-01' startTime='PT09H00M00S' endTime='PT18H30M00S'"

### Create a ReAct Agent with all three tools

In [117]:
# Create a react agent with all three tools
tools = [get_today, get_records, post_records]
react_agent = create_react_agent(model=llm, tools=tools)

# Stream the agent's actions and final response
# TODO Modify the user input if you want to test different scenarios
stream = react_agent.stream({"messages": ["Today I worked from 09:00 to 18:30. Please log this time entry into the system."]})
process_output(stream)

================================== Ai Message ==================================
Tool Calls:
  get_today (call_o76EHAO6U5NWxZIJ6hUWbdQ4)
 Call ID: call_o76EHAO6U5NWxZIJ6hUWbdQ4
  Args:
================================= Tool Message =================================
Name: get_today

2025-09-26
================================== Ai Message ==================================
Tool Calls:
  post_records (call_cmN6Es1N0yr4ylcgd94HmGdA)
 Call ID: call_cmN6Es1N0yr4ylcgd94HmGdA
  Args:
    data: {'startDate': '2025-09-26', 'startTime': 'PT09H00M00S', 'endTime': 'PT18H30M00S'}
    confirmation_message: Confirm work time from 09:00 to 18:30 on Friday, September 26th 2025.
================================= Tool Message =================================
Name: post_records

Entity created successfully: startDate='2025-09-26' startTime='PT09H00M00S' endTime='PT18H30M00S'
================================== Ai Message ==================================

Your work time from 09:00 to 18:30 on Friday, Sep

False

### Importance of Human-in-the-loop control

In [118]:
from IPython.display import Markdown
answer = react_agent.invoke({"messages": ["What issues could arrise due to missing HITL control with your current tooling? Keep it concise. Mention your tools."]})
display(Markdown(answer["messages"][-1].content))

Missing Human-In-The-Loop (HITL) control with the current tooling could lead to:

1. **Incorrect Time Records**: Tools like `functions.post_records` could post inaccurate work time records without human verification, leading to errors in time tracking.
2. **Data Overload**: Using `functions.get_records` without HITL might retrieve excessive or irrelevant data, making it difficult to manage and analyze.
3. **Automation Errors**: `multi_tool_use.parallel` could execute multiple tools simultaneously without human oversight, potentially causing conflicts or errors in data processing.

Overall, the absence of HITL control could result in data inaccuracies, inefficiencies, and potential conflicts in automated processes.

### New agent architecture: From ReAct Agent to HITL control
The prebuilt `create_react_agent` lacks Human-in-the-Loop (HITL) control for sensitive operations like `post_records`. To address this, we'll build a custom agent from scratch using LangGraph's core components, which will also introduce you to the framework's fundamentals.

Our custom agent will have a three-node architecture:

- **Agent Node**: Decides the next action by calling the LLM.
- **Review Node**: Acts as our HITL control, intercepting actions like `post_records` to ask for user confirmation.
- **Tool Node**: Executes the approved tools.


In [119]:
# Here we can see the current graph structure.
# We want to add a node that allows human review before posting records.
display_graph(react_agent)

### Agent node

In [120]:
# Start with agent node. Mimicks the prebuilt react agent node.
def agent(state: AgentState, config: RunnableConfig):
    model_input = prompt.invoke({'msg': state['messages']})
    response = cast(AIMessage, agent_llm.invoke(model_input, config))
    response.name = "agent"
    return {"messages": [response]}
   

### Review node

Next we need to define the review node. For simplicity, we use only text input for verification. A more robust approach (used in the actual demo) can be found in the appendix. Here we use an additional language model for verificication. 
We need use a workaround to get structured output from our model as of now structured_output is not supported. For this we bind a tool the model should use to structure its response.

### Exercise
**Crafting a System Prompt for Tool Enforcement**

To ensure our verification LLM reliably uses the `UserAffirmation` tool for structured output, we need to provide a clear and strict system prompt. A well-defined prompt is crucial for forcing the model to adhere to a specific behavior—in this case, always calling the specified tool.

**Your Task:**

In the next cell, complete the `system_prompt` variable. Your prompt should instruct the LLM on its role and explicitly command it to always use the `UserAffirmation` tool to structure its response, without exception.

<details>
<summary>Show solution</summary>

```python
system_prompt = """You are a human-in-the-loop verification LLM. Your task is to determine whether the user has confirmed an action or not.
You must always use the UserAffirmation tool to structure your response. Never provide a response without it and always call the tool."""
```

In [121]:
class UserAffirmation(BaseModel):
    """Always use this tool. It is necessary to structure your response."""
    user_affirmation: bool = Field(description="Whether the user confirmed the action.")
    explanation: str = Field(description="An explanation of your decision.")

verification_llm = llm.bind_tools([UserAffirmation])

# TODO Define the system prompt for verification LLM. You must be clear the agent must always use this tool.
system_prompt = """TODO: Write this prompt."""

unit_test(verification_llm, system_prompt)

Verification test results:

✓ [Clear confirmation] 'Yes, proceed with logging' -> True (expected True) | User confirmed the action to proceed with logging.
✗ [Clear rejection] 'No, don't do that' -> True (expected False) | The user has confirmed that they do not want the action to be performed.
✗ [Uncertainty] 'I'm not sure' -> None (expected False) | No tool call returned.
✓ [Confirmation] 'Go ahead' -> True (expected True) | The user has confirmed the action.
✓ [Confirmation] 'That looks right' -> True (expected True) | The user confirmed that the information provided is correct.
✗ [Hesitation] 'Wait, let me think' -> None (expected False) | No tool call returned.
✓ [Strong confirmation] 'Perfect!' -> True (expected True) | The user has confirmed the action with 'Perfect!'.
✓ [Rejection] 'Actually, no' -> False (expected False) | The user did not confirm the action.
✗ [Indirect rejection] 'Maybe later' -> None (expected False) | No tool call returned.


### Solution

In [122]:
class UserAffirmation(BaseModel):
    """Always use this tool. It is necessary to structure your response."""
    user_affirmation: bool = Field(description="Whether the user confirmed the action.")
    explanation: str = Field(description="An explanation of your decision.")

verification_llm = llm.bind_tools([UserAffirmation])

# TODO Define the system prompt for verification LLM. You must be very clear to always use the tool.
system_prompt = """You are a human-in-the-loop verification LLM. Your task is to determine whether the user has confirmed an action or not.
You must always use the UserAffirmation tool to structure your response. Never provide a response without it and always call the tool."""

unit_test(verification_llm, system_prompt)

Verification test results:

✓ [Clear confirmation] 'Yes, proceed with logging' -> True (expected True) | The user has confirmed the action by stating 'Yes, proceed with logging.'
✓ [Clear rejection] 'No, don't do that' -> False (expected False) | The user has clearly stated 'No, don't do that', indicating they do not want the action to be performed.
✓ [Uncertainty] 'I'm not sure' -> False (expected False) | The user expressed uncertainty and did not confirm the action.
✓ [Confirmation] 'Go ahead' -> True (expected True) | The user has confirmed the action by stating 'Go ahead'.
✓ [Confirmation] 'That looks right' -> True (expected True) | The user stated 'That looks right,' which indicates confirmation of the action.
✓ [Hesitation] 'Wait, let me think' -> False (expected False) | The user has requested more time to think and has not confirmed the action.
✓ [Strong confirmation] 'Perfect!' -> True (expected True) | The user used the word 'Perfect!' which indicates a positive confirmatio

### Review node

The `human_review` node intercepts `post_records` tool calls to request user confirmation via an `interrupt`. If no such calls are present, control passes directly to the `tools` node.

After the user responds to the interrupt, a verification LLM processes their input.
- **If approved**, execution proceeds to the `tools` node.
- **If denied**, a `ToolMessage` is added to the state, and control returns to the `agent` node.

**Key Points:**
- A `ToolMessage` is required for every tool call.
- After an `interrupt`, the `human_review` node re-executes from the beginning, which is a common pitfall to be aware of.

In [123]:
from typing import Union


def human_review(state: AgentState) -> Command[Literal["agent", "tools"]]:
    # Check if the last message contains a call to post_records
    last_message = state["messages"][-1]
    post_record_calls = [tool_call for tool_call in last_message.tool_calls if tool_call['name'] == 'post_records']

    # If there is a post_records call, ask for user confirmation
    if len(post_record_calls) > 0:

        # Get the confirmation message from the tool call arguments and ask the user to review
        confirmation_message = [call["args"]["confirmation_message"] for call in post_record_calls]
        user_review = interrupt({"task": "Review the action.",
                           "action": confirmation_message})
        
        # Use the verification LLM to determine if the user confirmed the action
        output = verification_llm.invoke(
            [('user', user_review), ('system', 'Verify whether the user wants to continue with the action.')])
        # Extract the user affirmation result
        should_continue = output.tool_calls[0]['args']['user_affirmation']
        print(f"Model explanation: {output.tool_calls[0]['args']['explanation']}")

        # If user confirmed, proceed to tools node, else go back to agent node
        if should_continue:
            # With Send we can select the next node and pass the current state
            return Send(node='tools', arg=state)
        else:
            # With Command we can select the next node and update the state
            return Command(update={"messages": [ToolMessage('User did not confirm action.', tool_call_id=call['id']) for call in post_record_calls]}, goto='agent')
    else:
        return Send(node='tools', arg=state)



### Exercise
**Determine the Next Node**

Understand the logic of the `human_review` node, decide which node the graph will transition to in each of the following scenarios. Fill in the blanks before revealing the answers.

**Your Task:**

1.  **Scenario A:**
    -   Last AI tool calls: `[get_records]`
    -   Next node: _______

2.  **Scenario B:**
    -   Last AI tool calls: `[post_records(...)]`
    -   User affirms (`should_continue = True`)
    -   Next node: _______

3.  **Scenario C:**
    -   Last AI tool calls: `[post_records(...)]`
    -   User declines (`should_continue = False`)
    -   Next node: _______

<details>
<summary>Show Answers</summary>

1.  **Scenario A** -> `tools`
2.  **Scenario B** -> `tools`
3.  **Scenario C** -> `agent`

</details>

### Tool Node

For the tools node we use the prebuilt ToolNode. It retrieves the tool calls from the last AIMessage and executes them. It provides a tool message for every tool call and appends it to the message history.

In [124]:
agent_tools = [post_records, get_today, get_records]
agent_llm = llm.bind_tools(agent_tools)
tool_node = ToolNode(agent_tools)

### Build the new agent and define control flow

Now we build our graph by adding the nodes and edges. Edges define what nodes to execute next.

In [125]:

def init_agent(tool_node):
    workflow = StateGraph(AgentState)

    workflow.add_node('agent', agent)
    workflow.add_node('tools', tool_node)
    workflow.add_node('human_review', human_review)

    workflow.add_edge(START, "agent")
    workflow.add_edge("tools", "agent")
    return workflow

workflow = init_agent(tool_node)

#Mermaid API is experiencing issues currently. Hotfix
# display(workflow.compile())

# Here you can see the current graph structure with the new human_review node.
# As you can see the agent node is not connected to anything yet.
display_graph(workflow.compile())

### Define control flow with Conditional edges

Conditional edges define what node to execute next based on a condition. We add a conditional edge which routes the execution from the agent node either to the review node or the end node depending on whether the language model executed a tool call. 

In [126]:
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0:
        return "tool call"
    return "__end__"

workflow.add_conditional_edges(source="agent", path=should_continue, path_map={"tool call": "human_review", "__end__": END})


# display(workflow.compile())
display_graph(workflow.compile())

Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


### Finalize by compiling the graph

Finally, we compile our graph. When compiling we add a checkpointer to achieve thread level persistence. With a checkpointer specified at compilation, a snapshot of the graph state is saved at every superstep. This is crucial for human-in-the-loop interactions as we need to resume execution after an interrupt is called. 



In [127]:

checkpointer = MemorySaver()
timesheet_agent = workflow.compile(checkpointer=checkpointer)
display_graph(timesheet_agent)


### Add a system prompt

In order to ensure consistent and useful behaviour we need to define a well structured system prompt next. 
Generally, you should:
  1. Define the role and persona.
  2. Establish context and objectives
  3. Outline clear instructions and constraints
  4. Provide examples of ideal responses (Optional)


By utilizing few-shot prompting model performance can be hugely improved. It also makes sense to encourage iterative clarification.

In [128]:
system_prompt =f"""
Role and Objective:
  - You are a helpful AI Agent dedicated to assisting users with time recording.
  - Your primary tasks include retrieving and posting timesheet data based on user requests.

Responsibilities:
  - Logging Work Time: Only log actual work time. Do not include any breaks.
    - User: Today I worked from 6 to 16 with a half hour break at 12. -> You should: Log time from 6 to 12 and from 12:30 to 16.
  - Data Handling: When posting records, execute as many post_records calls in parallel as possible using the provided information.
  - When calling a tool always describe the step you are taking in the message containing the tool call.

Interaction Guidelines:
  - The user's most recent input always takes precedence over older input.
  - Do not ask the user for confirmation.
  - Always provide a confirmation_message in the tool call when posting records.
  - Language Consistency: Always respond in the same language as the user.
  - If the user’s request is too ambiguous (for example, very unclear work times), ask clarifying questions rather than making farfetched assumptions.
  - Use your tools to retrieve the exact dates.
  - If the user mentions a specific week and weekday use your tools to infer the date. Never ask the user for the date in this case.
"""

prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    MessagesPlaceholder(variable_name='msg')
])

### Test out the agent

Now we can stream the output of our agent. We hand over a dictionary containing the user's input and a config with our thread id. Each thread represents an individual session betweeen the graph and the user. So, if we want to continue our conversation, we need to pass the same thread id to the graph.

In [129]:

config = {"configurable": {"thread_id": str(uuid.uuid1())}}

user_input = {'messages': ['user', 'Today, I worked from 6 to 6 with a half hour break at 12.']}
process_output(timesheet_agent.stream(user_input, config, stream_mode='updates'))

user_input = Command(resume='Sure.')
process_output(timesheet_agent.stream(user_input, config, stream_mode='updates'))




================================== Ai Message ==================================
Name: agent
Tool Calls:
  get_today (call_adzIlDF6kSqNae0Sw0zisPWb)
 Call ID: call_adzIlDF6kSqNae0Sw0zisPWb
  Args:
================================= Tool Message =================================
Name: get_today

2025-09-26
================================== Ai Message ==================================
Name: agent
Tool Calls:
  post_records (call_bu6bw60UktFxiSAES1SyhtYX)
 Call ID: call_bu6bw60UktFxiSAES1SyhtYX
  Args:
    data: {'startDate': '2025-09-26', 'startTime': 'PT06H00M00S', 'endTime': 'PT12H00M00S'}
    confirmation_message: Confirm work time from 06:00 to 12:00 on Friday, September 26th 2025.
  post_records (call_a8XwSxbLXnk4KlKP6c2uv5kZ)
 Call ID: call_a8XwSxbLXnk4KlKP6c2uv5kZ
  Args:
    data: {'startDate': '2025-09-26', 'startTime': 'PT12H30M00S', 'endTime': 'PT18H00M00S'}
    confirmation_message: Confirm work time from 12:30 to 18:00 on Friday, September 26th 2025.
Interrupt(value={'task'

False

### Interact with the agent

Now you can try interacting with the agent.

In [130]:

config = {"configurable": {"thread_id": str(uuid.uuid1())}}
interrupted = False

print("Type to interact with the agent (type q to quit):\n")
while True:
    user_input = input()
    if user_input.lower() == 'q':
        break
    print(user_input)

    if interrupted:
        interrupted = False
        user_input = Command(resume=user_input)
    else:
        user_input = {'messages': ['user', user_input]}

    interrupted = process_output(timesheet_agent.stream(user_input, config, stream_mode="updates"))
      



Type to interact with the agent (type q to quit):

hi
================================== Ai Message ==================================
Name: agent

Hello! How can I assist you today?


today from 8 to 4
================================== Ai Message ==================================
Name: agent
Tool Calls:
  get_today (call_nuxFyFNEeoXdUN5qIb9sAr63)
 Call ID: call_nuxFyFNEeoXdUN5qIb9sAr63
  Args:
================================= Tool Message =================================
Name: get_today

2025-09-26
================================== Ai Message ==================================
Name: agent
Tool Calls:
  post_records (call_DAyJHNDaSMUWEBY72IRjPyhI)
 Call ID: call_DAyJHNDaSMUWEBY72IRjPyhI
  Args:
    data: {'startDate': '2025-09-26', 'startTime': 'PT08H00M00S', 'endTime': 'PT16H00M00S'}
    confirmation_message: Confirm work time from 08:00 to 16:00 on Friday, September 26th 2025.
Interrupt(value={'task': 'Review the action.', 'action': ['Confirm work time from 08:00 to 16:00 on Fri

# Extension with MCP

## Get MCP tools

### Retrieve required tokens

In [131]:
# Extension with MCP / Community Tool Kits

# Retrieve MCP token from environment variable or use default for demo purposes
MCP_TOKEN = os.environ.get("MCP_TOKEN", None)
if MCP_TOKEN is None:
    raise ValueError("MCP_TOKEN environment variable is not set.")
url = os.environ.get("MCP_API_BASE_URL", None)
if url is None:
    raise ValueError("MCP_API_BASE_URL environment variable is not set.")

In [132]:
# Define MCP server configurations
def http_mcp_server(url):
    # url = url.rstrip("/")
    return {
        "email": {
            "transport": "streamable_http",
            "url": url,
            "headers": {"Authorization": f"Bearer {MCP_TOKEN}"}
        }
    }

In [133]:
# Initialize MCP client and load email tools
mcp = MultiServerMCPClient(http_mcp_server(url))
email_tools = await mcp.get_tools(server_name="email")

### Update agent

In [134]:
# Combine with existing tools
tools = [get_today, get_records, post_records, *email_tools]
print(f"Loaded tools from MCP server. Available tools: {[tool.name for tool in email_tools]}")
agent_llm = llm.bind_tools(tools)

Loaded tools from MCP server. Available tools: ['search_messages', 'get_message']


In [135]:
# Reinitialize workflow with new tools
tools_node = ToolNode(tools)
workflow = init_agent(tool_node=tools_node)
workflow.add_conditional_edges(
    source="agent",
    path=should_continue,
    path_map={"tool call": "human_review", "__end__": END}
)

# Finalize agent with checkpointer to retain state across sessions
new_agent  = workflow.compile(checkpointer=checkpointer)

### Test out the new MCP capabilities

In [136]:
# Test the agent with email tools
config = {"configurable": {"thread_id": str(uuid.uuid1())}}
answer = await new_agent.ainvoke({"messages": ["Please check my email inbox."]}, config)
for msg in answer["messages"][:-1]:
    msg.pretty_print()

display(Markdown(answer["messages"][-1].content))

================================ Human Message =================================

Please check my email inbox.
================================== Ai Message ==================================
Name: agent
Tool Calls:
  search_messages (call_Mi9dXVmRYrwT1kUPmUmjUw7m)
 Call ID: call_Mi9dXVmRYrwT1kUPmUmjUw7m
  Args:
    limit: 10
================================= Tool Message =================================
Name: search_messages

[{"id":"1994d12b9574738a","thread_id":"1994d12b9574738a","subject":"Workshop Notes","from":"\"Abdulla, Can\" <can.abdulla@sap.com>","date":"Mon, 15 Sep 2025 11:11:30 +0000","preview_text":"Hi Alice,\r\n\r\nThanks for the workshop today from 14:00–16:00 with our team.\r\nPlease send the notes wh..."}]


You have one email in your inbox:

- **Subject:** Workshop Notes
- **From:** "Abdulla, Can" <can.abdulla@sap.com>
- **Date:** Mon, 15 Sep 2025 11:11:30 +0000
- **Preview:** Hi Alice, Thanks for the workshop today from 14:00–16:00 with our team. Please send the notes wh...

Would you like to read the full email?